# Canopy simulation tutorial

This notebook contains sample cases for the Eradiate preview session of Dec 7th 2021. In this tutorial, we introduce the canopy simulation capabilities of Eradiate. We will explore the features and interfaces of different objects used to construct a scene, simulate radiative transfer and visualise results.

In [ ]:
# We load the Rich notebook extension for improved object inspection
%load_ext rich

# We set the Matplotlib inline backend to SVG for better figure quality
from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats("svg")

# We import general processing and plotting libraries
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

# We import the top-level Eradiate module
import eradiate

# For convenience, we alias a few components
from eradiate import unit_registry as ureg
import eradiate.scenes as ertsc
import eradiate.experiments as ertxp

# We'll use the monochromatic mode in double precision
eradiate.set_mode("mono_double")

# We activate a few convenient presets
eradiate.notebook.install()
eradiate.plot.set_style(rc={"figure.dpi": 96})

We start with the definition of a couple of utility functions which will simplify the visualisation of results.

In [ ]:
def show_camera(
    exp,
    measure_id,
    robust=True,
    add_colorbar=False,
    vmin=None,
    vmax=None
):
    """
    Display the output of a monochromatic camera measure.
    """
    _, ax = plt.subplots(1,1)
    exp.results[measure_id]["radiance"].squeeze(drop=True).plot.imshow(
        ax=ax,
        origin="upper",
        cmap="Greys_r",
        vmin=vmin,
        vmax=vmax,
        robust=robust,
        add_colorbar=add_colorbar,
    )
    ax.set_aspect(1)
    plt.show()
    plt.close()

def show_brf(exp, measure_id):
    """
    Display the BRF output of a distant radiance measure.
    """
    _, ax = plt.subplots(1,1)
    with plt.style.context({"lines.linestyle": ":", "lines.marker": "."}):
        exp.results[measure_id]["brf"].squeeze(drop=True).plot(ax=ax, x="vza")
    plt.show()
    plt.close()

## Rendering a surface


First, let's create a very simple scene consisting of a default Lambertian surface observed by a camera. We start by defining the surface and set its width to 10 m (the default is much larger).

In [ ]:
lambertian_surface=ertsc.surface.LambertianSurface(
    width=10 * ureg.m, reflectance=0.5
)

Now, let's make a monochromatic render of this scene. We'll produce a 4:3 ratio image with a fairly low resolution sample count—all we want is to visualise the scene.

**Note**: Using the correlated *k* band mode to render scenes is more computationally intensive. Therefore, we will stick to Eradiate's monochromatic mode for renders in this tutorial.

In [ ]:
camera_oblique = ertsc.measure.PerspectiveCameraMeasure(
    id="camera_oblique",
    origin=[15, 15, 15] * ureg.m,
    target=[0, 0, 0] * ureg.m,
    up=[0, 0, 1],
    film_resolution=(320, 240),
    spp=512,
)
exp = ertxp.RamiExperiment(
    surface=lambertian_surface, measures=camera_oblique
)

In [ ]:
exp.run()
show_camera(exp, "camera_oblique")

You might notice rendering artifacts. This is because Eradiate's camera settings are currently not optimised for rendering. This will be improved in future releases.

## Rendering a surface and canopy


Let's add a canopy above this ground patch. We'll configure a homogeneous canopy composed of randomly oriented floating disks. This is done using the [`DiscreteCanopy` class](https://eradiate.readthedocs.io/en/latest/rst/reference/generated/autosummary/eradiate.scenes.biosphere.DiscreteCanopy.html), or, rather, its [`homogeneous()`](https://eradiate.readthedocs.io/en/latest/rst/reference/generated/autosummary/eradiate.scenes.biosphere.DiscreteCanopy.html) class constructor. We will use abstract parameters for the sake of demonstrating this feature:

In [ ]:
homogeneous_canopy = ertsc.biosphere.DiscreteCanopy.homogeneous(
    l_vertical=1.0 * ureg.m,
    l_horizontal=10.0 * ureg.m,
    lai=2.0,
    leaf_radius=10 * ureg.cm,
)

We add this canopy to a scene with a surface using the [`RamiExperiment` class](https://eradiate.readthedocs.io/en/latest/rst/reference/generated/autosummary/eradiate.experiments.RamiExperiment.html). Note that this time, we don't set the surface size: it is automatically adjusted to match the extents we set for the canopy object.

In [ ]:
exp = ertxp.RamiExperiment(
    surface=ertsc.surface.LambertianSurface(reflectance=0.5),
    canopy=homogeneous_canopy,
    measures=camera_oblique,
)
exp.run()
show_camera(exp, "camera_oblique")


## Rendering a surface and padded canopy


In order to simulate the fact that our canopy is periodic, we can pad it with clones of itself. Let's start with just of few of them.

**Tip**: If the render time is too long, you can reduce the number of radiance samples per image pixel by adjusting the measure's `spp` parameter.


In [ ]:
camera_oblique = ertsc.measure.PerspectiveCameraMeasure(
    id="camera_oblique",
    origin=[15, 15, 15] * ureg.m,
    target=[0, 0, 0],
    up=[0, 0, 1],
    film_resolution=(320, 240),
    spp=64,  # Lower sample count value ensures quicker rendering
)

exp = ertxp.RamiExperiment(
    surface=ertsc.surface.LambertianSurface(reflectance=0.5),
    canopy=homogeneous_canopy,
    padding=1,
    measures=camera_oblique,
)
exp.run()
show_camera(exp, "camera_oblique")

Now, let's relocate our camera to the top of canopy. We will observe it at a grazing angle.

In [ ]:
camera_toc = ertsc.measure.PerspectiveCameraMeasure(
    id="camera_toc",
    origin=[15, 15, 2] * ureg.m,
    target=[0, 0, 0],
    up=[0, 0, 1],
    film_resolution=(320, 240),
    spp=64,  # Lower sample count value ensures quicker rendering
)
exp = ertxp.RamiExperiment(
    surface=ertsc.surface.LambertianSurface(reflectance=0.5),
    canopy=homogeneous_canopy,
    padding=2,
    measures=[camera_toc],
)
exp.run()
show_camera(exp, "camera_toc")

## Computing the BRF in the principal plane

Now that we know how to build our scene, we are going to compute the BRF in the principal plane for a similar scene. This is similar to the test cases for which we can submit results to the RAMI online checker (ROMC.)

In order to observe something meaningful, let's set the Sun zenith angle to 30°.

In [ ]:
illumination = ertsc.illumination.DirectionalIllumination(
    zenith=30.0, azimuth=0.0
)

brf_pplane = ertsc.measure.MultiDistantMeasure.from_viewing_angles(
    id="brf_pplane",
    azimuths=0.0,
    zeniths=np.arange(-75, 76, 5),
    spp=50000,
)

exp = ertxp.RamiExperiment(
    illumination=illumination,
    surface=ertsc.surface.LambertianSurface(),
    canopy=homogeneous_canopy,
    padding=10,
    measures=[brf_pplane],
)
exp.run()
show_brf(exp, "brf_pplane")

There still is some Monte Carlo noise, but we're keeping the number of samples low in order so that the simulation remains short.

**Exercises for the interested reader**

1. Change the viewing zenith angle and verify that the hot spot moves accordingly.
2. Change the viewing azimuth angle and adjust the measure so as to keep it aligned with the principal plane.
3. Compute the BRF in a plane orthgonal to the principal plane.
4. Gradually increase padding and determine from which value the quasi-periodic approximation can be considered valid.

## Computing the BRF in the principal plane with an atmosphere

Finally, we can combine our canopy with an atmosphere. This uses a different experiment class ([`Rami4ATMExperiment`](https://eradiate.readthedocs.io/en/latest/rst/reference/generated/autosummary/eradiate.experiments.Rami4ATMExperiment.html)), which merges the capabilities of the `RamiExperiment` and `OneDimExperiment` classes. We will only use a purely scattering atmosphere and perform monochromatic simulations so as to limit the runtime of our examples.

**Note**: This section demonstrates experimental features. We are aware of several issues, notably intersection inaccuracies. This will be fixed in the coming weeks!

In [ ]:
eradiate.set_mode("mono_double")

We will first add a default atmosphere and reuse the top-of-canopy view from our earlier example.

In [ ]:
camera_toc = ertsc.measure.PerspectiveCameraMeasure(
    id="camera_toc",
    origin=[15, 15, 2] * ureg.m,
    target=[0, 0, 0],
    up=[0, 0, 1],
    film_resolution=(320, 240),
    spp=64,  # Lower sample count value ensures quicker rendering
)
exp = ertxp.Rami4ATMExperiment(
    surface=ertsc.surface.LambertianSurface(reflectance=0.5),
    atmosphere=ertsc.atmosphere.MolecularAtmosphere(has_absorption=False),
    canopy=homogeneous_canopy,
    padding=2,
    measures=[camera_toc],
)
exp.run()
show_camera(exp, "camera_toc")

Our render is very noisy, but we can clearly see that the addition of the atmosphere to our previous setup added a haze. We can now reposition our camera to the ToA and select a wavelength in the UV, where Rayleigh scattering is strong:

In [ ]:
for atmosphere in [
    None,  # Atmosphere-free reference
    ertsc.atmosphere.MolecularAtmosphere(has_absorption=False)  # Rayleigh scattering only (no gaseous absorption, no aerosols)
]:
    camera_toa = ertsc.measure.PerspectiveCameraMeasure(
        id="camera_toa",
        origin=[0, 0, 101] * ureg.km,
        target=[0, 0, 0],
        up=[0, 1, 0],
        film_resolution=(128, 128),
        fov=2e-2,  # The camera is far away, reducing the field of view is like zooming
        spp=512,  # Lower sample count value ensures quicker rendering
        spectral_cfg={"wavelengths": [300.0] * ureg.nm},
    )
    exp = ertxp.Rami4ATMExperiment(
        surface=ertsc.surface.LambertianSurface(reflectance=0.5),
        atmosphere=atmosphere,
        canopy=homogeneous_canopy,
        padding=2,
        measures=[camera_toa],
    )
    exp.run()
    show_camera(exp, "camera_toa", add_colorbar=True, vmin=0.07, vmax=0.15)
    # Note: Normalise v-range so that we can compare both images

Despite the noise, we can see that Rayleigh scattering dims the signal at this wavelength.

**Exercises for the interested reader**

1. With a non-zenith illumination, set up a TOA BRF measurement for a few wavelengths ranging from the UV to NIR regions and see how Rayleigh scattering broadens the hot spot reflective lobe.
1. Increase padding value and see how adjacency effects impact simulation results differently depending on the wavelength.
2. Switch to the CKD mode and compute the TOA BRF for Sentinel-2 bands 1 and 8A. You will have to use the AFGL 1986 atmosphere model. You will also have to adust simulation parameters to keep the computational time reasonable while setting up your simulation (decrease the sample count while preparing it, then increase it to compute the final results).

In [ ]:
# Solution to exercise 1

toa_brf = ertsc.measure.MultiDistantMeasure.from_viewing_angles(
    id="toa_brf",
    zeniths=np.arange(-75, 76, 5),
    azimuths=0.0,
    spp=10000,
    spectral_cfg={"wavelengths": [300, 325, 350, 400, 450, 500]}
)

exp = ertxp.Rami4ATMExperiment(
    illumination=ertsc.illumination.DirectionalIllumination(zenith=30.0),
    surface=ertsc.surface.LambertianSurface(reflectance=0.5),
    atmosphere=ertsc.atmosphere.MolecularAtmosphere(has_absorption=False),
    canopy=homogeneous_canopy,
    padding=2,
    measures=[toa_brf],
)
exp.run()
ds = exp.results["toa_brf"]

with plt.style.context({"lines.linestyle": ":", "lines.marker": "."}):
    ds.brf.squeeze(drop=True).plot(hue="w", x="vza")
plt.show()
plt.close()

In [ ]:
# Solution to exercise 3

eradiate.set_mode("ckd_double")
eradiate.config.progress = 1  # Do not show Mitsuba-level progress bar

toa_brf = ertsc.measure.MultiDistantMeasure.from_viewing_angles(
    id="toa_brf",
    zeniths=np.arange(-75, 76, 5),
    azimuths=0.0,
    spp=1000,
    spectral_cfg={"srf": "sentinel_2a-msi-8a"}
)

exp = ertxp.Rami4ATMExperiment(
    illumination=ertsc.illumination.DirectionalIllumination(zenith=30.0),
    surface=ertsc.surface.LambertianSurface(reflectance=0.5),
    atmosphere=ertsc.atmosphere.MolecularAtmosphere.afgl_1986(),
    canopy=homogeneous_canopy,
    padding=2,
    measures=[toa_brf],
)
exp.run()
ds = exp.results["toa_brf"]

with plt.style.context({"lines.linestyle": ":", "lines.marker": "."}):
    ds.brf_srf.squeeze(drop=True).plot(x="vza")
plt.show()
plt.close()